# Noobai-XL-1.1 Text-to-Image Generator

This notebook implements the Noobai-XL-1.1 model with various customization options including:
- Different sampler methods
- Adjustable sampling steps
- Custom seeds
- CFG scale control
- Image size control
- HiRes fix with 8x-NKMD-Superscale

In [ ]:
# Install required packages
!pip install -q diffusers==0.21.4 transformers==4.31.0 accelerate==0.21.0 xformers
!pip install opencv-python basicsr facexlib gfpgan
!pip install -q gradio

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from diffusers import DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler
import cv2
import numpy as np
from PIL import Image
import gradio as gr

# Initialize the model
model_id = "Laxhar/noobai-XL-1.1"

# Create pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None
)

# Move to GPU and enable memory efficient attention
pipe = pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

# Remove token length limit
pipe.tokenizer.model_max_length = 1000000

# Dictionary of available samplers
samplers = {
    "DPM++ 2M": DPMSolverMultistepScheduler,
    "DDIM": DDIMScheduler,
    "LMS": LMSDiscreteScheduler,
    "Euler": EulerDiscreteScheduler,
    "Euler a": EulerAncestralDiscreteScheduler
}

def change_sampler(pipe, sampler_name):
    if sampler_name in samplers:
        pipe.scheduler = samplers[sampler_name].from_config(pipe.scheduler.config)
    return pipe

def superscale_image(img, scale_factor=4, creativity=1.0):
    # Convert PIL Image to numpy array
    img_np = np.array(img)
    
    # Calculate new dimensions
    height, width = img_np.shape[:2]
    new_height = int(height * scale_factor)
    new_width = int(width * scale_factor)
    
    # Apply 8x-NKMD-Superscale-like upscaling
    # Using a combination of Lanczos and detail enhancement
    upscaled = cv2.resize(img_np, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
    
    # Enhance details based on creativity parameter
    if creativity > 0:
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) * creativity
        enhanced = cv2.filter2D(upscaled, -1, kernel)
        upscaled = cv2.addWeighted(upscaled, 0.7, enhanced, 0.3, 0)
    
    return Image.fromarray(upscaled)

def generate_image(prompt, negative_prompt, sampler_name, steps, cfg_scale, seed, 
                  width, height, use_hires_fix, hires_scale, hires_creativity):
    # Set seed for reproducibility
    if seed == -1:
        seed = np.random.randint(0, 1000000)
    generator = torch.Generator("cuda").manual_seed(seed)
    
    # Change sampler
    pipe = change_sampler(pipe, sampler_name)
    
    # Generate image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=steps,
        guidance_scale=cfg_scale,
        width=width,
        height=height,
        generator=generator
    ).images[0]
    
    # Apply HiRes fix if selected
    if use_hires_fix:
        image = superscale_image(image, hires_scale, hires_creativity)
    
    return image, seed

# Create Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", lines=3)
            negative_prompt = gr.Textbox(label="Negative Prompt", lines=2)
            sampler = gr.Dropdown(choices=list(samplers.keys()), value="DPM++ 2M", label="Sampler")
            steps = gr.Slider(minimum=1, maximum=150, value=30, step=1, label="Sampling Steps")
            cfg = gr.Slider(minimum=1, maximum=20, value=7, step=0.5, label="CFG Scale")
            seed = gr.Number(value=-1, label="Seed (-1 for random)")
            width = gr.Slider(minimum=256, maximum=1024, value=512, step=64, label="Width")
            height = gr.Slider(minimum=256, maximum=1024, value=512, step=64, label="Height")
            
            with gr.Accordion("HiRes Fix Settings", open=False):
                use_hires = gr.Checkbox(label="Enable HiRes Fix", value=False)
                hires_scale = gr.Slider(minimum=1, maximum=4, value=2, step=0.5, label="Scale Factor")
                hires_creativity = gr.Slider(minimum=0, maximum=2, value=1, step=0.1, label="Detail Enhancement")
            
            generate_btn = gr.Button("Generate")
        
        with gr.Column():
            output_image = gr.Image(label="Generated Image")
            used_seed = gr.Number(label="Used Seed", interactive=False)
    
    generate_btn.click(
        fn=generate_image,
        inputs=[prompt, negative_prompt, sampler, steps, cfg, seed, 
                width, height, use_hires, hires_scale, hires_creativity],
        outputs=[output_image, used_seed]
    )

demo.launch(share=True)